In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output

from aicsimageio.readers import CziReader
from czifile import CziFile
from tifffile import imread, imwrite

import skimage.filters as filters

from skimage.segmentation import slic, join_segmentations, watershed
from skimage.morphology import remove_small_objects, remove_small_holes, skeletonize, convex_hull_image
from skimage.measure import label
import skimage.measure as measure
from skimage.filters import threshold_otsu

import h5py
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import napari
import numpy as np
import numpy.ma as ma
import pandas as pd
import subprocess 
import os, sys

import tkinter as tk
from tkinter import filedialog


In [4]:
%gui qt

In [5]:
XYZ_SCALE = (0.156, 0.156, 1) 

In [15]:
# Create a root window (it won't be displayed)
root = tk.Tk()
root.withdraw()

# Open a file dialog for selecting a file
file_path = filedialog.askopenfilename()

# Check if a file was selected
if file_path:
    print(f"Selected Label File: {file_path}")
else:
    print("No file selected.")

Selected Label File: /mnt/5404b8a5-71b7-4464-9a1e-b40cd26fac58/Data_Drive/Wissam/Retina_Project/Dataset_Images/2018-0153_OS_Pie8_72F_mid-peri_Z-stack_(1)_glia.npy


In [16]:
# Microglia labels
gliaLabels =  np.load(file_path)
updatedLabels = gliaLabels.copy()

In [11]:
try:
    # Generate a temp tiff to segment
    imagepath = os.path.basename(file_path)[:-9]+'.czi'
    retina = CziFile(imagepath)
    images = retina.asarray()
    ch0 = images[0, 0, 0, :, :, : ]
except:
    # Create a root window (it won't be displayed)
    root = tk.Tk()
    root.withdraw()

    # Open a file dialog for selecting a file
    imagepath = filedialog.askopenfilename()
    retina = CziFile(imagepath)
    images = retina.asarray()
    ch0 = images[0, 0, 0, :, :, :, 0]
    CD68 = images[0, 0, 3, :, :, :, 0]
    # Check if a file was selected
    if file_path:
        print(f"Selected Label File: {imagepath}")
    else:
        print("No file selected.")

Selected Label File: /mnt/5404b8a5-71b7-4464-9a1e-b40cd26fac58/Data_Drive/Wissam/Retina_Project/Dataset_Images/2018-0153 OS Pie8 72F mid-peri Z-stack (1)(1).czi


In [12]:
viewer = napari.Viewer()
viewer.add_image(ch0, scale=[XYZ_SCALE[2], XYZ_SCALE[0], XYZ_SCALE[1]] )
viewer.add_labels(updatedLabels, scale=[XYZ_SCALE[2], XYZ_SCALE[0], XYZ_SCALE[1]] )


# Create a text input widget
text_input = widgets.Text(
    value='',  # Initial value of the input field
    placeholder='Enter a comma-separated list of integers',
    description='Input:',
    disabled=False
)

# Function to handle text input and convert to a list of integers
def process_input(change):
    input_text = change['new']
    try:
        # Split the input text by commas and convert to integers
        integer_list = [int(x.strip()) for x in input_text.split(',')]
        output_label.value = f"List of Integers: {integer_list}"
        return integer_list
    except ValueError:
        output_label.value = "Invalid input. Please enter a valid comma-separated list of integers."

# Register the function to be called when the text input changes
_labelInts = text_input.observe(process_input, names='value')

# Create an output label widget
output_label = widgets.Label(
    value="List of Integers: []"
)

# Display the widgets
display(text_input)
display(output_label)

Text(value='', description='Input:', placeholder='Enter a comma-separated list of integers')

Label(value='List of Integers: []')

In [ ]:

# Create a button widget
button = widgets.Button(description="Remove Labels")


# Function to execute code when the button is clicked
intList= text_input.value
def execute_code(b):
    integer_list = [int(x.strip()) for x in intList.split(',')]
    for _label in integer_list:
        print('Label {} removed'.format(_label))
        
        updatedLabels[updatedLabels==_label]=0
    viewer.layers['updatedLabels'].data = updatedLabels
# Connect the button's click event to the execute_code function
button.on_click(lambda b: execute_code(b))

# Display the widgets
display(button)



In [17]:
def returnConvexHullVolume(labels, label, xyz):
    volex_units = xyz[0] * xyz[1] * xyz[2]
    hull = np.zeros_like(labels)
    _temp =  np.zeros_like(labels)
    _temp[labels==label] =1
    for _z in range(_temp.shape[0]):
        if np.sum(_temp[_z, : , :])>0:
            hull[_z, :, : ] = convex_hull_image(_temp[_z, : , :])

    convex_volume = np.sum(hull*volex_units)
    return convex_volume

def returnCD86levels(cd86_ch, microglia, threshold, xyz):
    '''
    Input is the cd86 channel and microglia
    
    Returns the intensity of the stain in the bounds of the microglia 
    '''
    # Apply the mask to the image
    masked_image = cd86_ch[microglia==1]

    # Calculate maximum and average values
    max_value = np.max(masked_image)
    average_value = np.mean(masked_image)
    positive = max_value >= threshold
    
    
    threshold_cd86 = cd86_ch.copy()
    threshold_cd86 = threshold_cd86*microglia
    threshold_cd86[threshold_cd86 < threshold] = 0
    threshold_cd86[threshold_cd86 > 0 ] = 1
    
    voxels = np.sum(threshold_cd86) * xyz[0] * xyz[1] * xyz[2]
    area = np.sum(np.max(threshold_cd86, axis=0))* xyz[0] * xyz[1]
    volume_ratio = voxels / (np.sum(microglia)* xyz[0] * xyz[1] * xyz[2])
    area_ratio = area / (np.sum(np.max(microglia, axis=0))* xyz[0] * xyz[1])
    

    return max_value, average_value, positive, voxels, area, volume_ratio, area_ratio

def find_bounding_box(data, label):

    rows, cols = np.where(data == label)

    if rows.size == 0 or cols.size == 0:
        return None, None, None, None

    min_y, max_y = np.min(rows), np.max(rows)
    min_x, max_x = np.min(cols), np.max(cols)

    return min_y, min_x, max_y, max_x   

def majorminoraxix(labelArray, localLabel=1):
    zProjection = np.max(labelArray, axis=0)
    y1, x1, y2, x2 = find_bounding_box(zProjection, localLabel)
    
    yAxisLen = y2-y1
    xAxisLen = x2-x1
    
    if yAxisLen > xAxisLen:
        return xAxisLen, yAxisLen, xAxisLen/yAxisLen
    else:
        return yAxisLen, xAxisLen, yAxisLen/xAxisLen


    
    
def returnMeasurements(microglia_labels, label_num, xyz, cd86_ch, threshold, showglia=False):
    # Make a binary copy of the microglia
    microglia = microglia_labels.copy()
    microglia[microglia!=label_num] = 0
    microglia[microglia!=0] = 1

    basic_metrics = {}
    
    volume = np.sum(microglia)*xyz[0]*xyz[1]*xyz[2]
    microglia2D = np.max(microglia, axis=0)
    if showglia == True:
        print("Displaying Microglia {}".format(label_num))
        plt.imshow(microglia2D)
        plt.show()

    area = np.sum(microglia2D) * xyz[0] * xyz[1]
    perimeter = measure.perimeter(microglia2D)* xyz[0] 
    data = measure.regionprops(microglia2D)
    minorAxisLen, majorAxisLen, axisRatio = majorminoraxix(microglia, 1)


    basic_metrics['Microglia_ID'] = int(label_num)
    basic_metrics['Cell_Volume'] = volume
    basic_metrics['Cell_Area'] = area
    basic_metrics['Cell_Perimeter'] = perimeter
    basic_metrics['Convex_Hull_Area'] = data[0]['area_convex'] * xyz[0] * xyz[1]
    basic_metrics['Convex_Hull_Perimeter'] = measure.perimeter(data[0]['image_convex'])* xyz[0]
    basic_metrics['Cell_Solidity'] = basic_metrics['Cell_Area']/basic_metrics['Convex_Hull_Area']
    basic_metrics['Convex_Hull_Volume'] = returnConvexHullVolume(microglia_labels, label_num, xyz)
    basic_metrics['Cell_Convexity'] = basic_metrics['Convex_Hull_Perimeter']/basic_metrics['Cell_Perimeter']
    basic_metrics['Cell_Circularity'] = (4*math.pi*basic_metrics['Cell_Area'])/(basic_metrics['Cell_Perimeter']**2)    
    basic_metrics['CD68_Max'], basic_metrics['CD68_Mean'], basic_metrics['CD68_Positive'], basic_metrics['CD68_Volume'], basic_metrics['CD68_Area'], basic_metrics['CD68_Volume_Ratio'], basic_metrics['CD68_Area_Ratio'] = returnCD86levels(cd86_ch, microglia, threshold, xyz)
    basic_metrics['Minor_Axis'], basic_metrics['Major_Axis'], basic_metrics['Axis_Ratio'] = xyz[0]*minorAxisLen, xyz[0]*majorAxisLen, axisRatio

    
    return basic_metrics
    
    


In [18]:
# Get image file name
ref_file = os.path.basename(file_path)
ref_file = ref_file[:-4]
ref_file = ref_file.replace(" ", "_")
print(ref_file)
np.save(ref_file+"_glia_clean.npy", updatedLabels)

# Dataframe to store results
basic_measures_datatable = pd.DataFrame()

# Make list of label keys and remove background label
detected_glia = list(np.unique(updatedLabels))
detected_glia.pop(0)

inclusion = np.percentile(CD68, 99.95)


# Return basic measures for each microglia and add them to the dataframe
for glia_int in detected_glia:
    basic_measures_datatable = pd.concat([basic_measures_datatable, pd.DataFrame.from_records([returnMeasurements(updatedLabels, glia_int, XYZ_SCALE, CD68, inclusion)])])

# Reindex dataframe
basic_measures_datatable.reset_index(drop=True, inplace=True)

# Add file used
basic_measures_datatable['Ref_Image'] = ref_file

display(basic_measures_datatable)

#Save results to CSV
basic_measures_datatable.to_csv("{}_clean_basic_measure.csv".format(ref_file))

2018-0153_OS_Pie8_72F_mid-peri_Z-stack_(1)_glia


,Microglia_ID,Cell_Volume,Cell_Area,Cell_Perimeter,Convex_Hull_Area,Convex_Hull_Perimeter,Cell_Solidity,Convex_Hull_Volume,Cell_Convexity,Cell_Circularity,...,CD68_Mean,CD68_Positive,CD68_Volume,CD68_Area,CD68_Volume_Ratio,CD68_Area_Ratio,Minor_Axis,Major_Axis,Axis_Ratio,Ref_Image
0,1,1715.882688,321.454224,236.247857,653.056560,103.186650,0.492230,5161.008528,0.436773,0.072376,...,2700.619192,True,1.557504,0.997776,0.000908,0.003104,31.512,31.824,0.990196,2018-0153_OS_Pie8_72F_mid-peri_Z-stack_(1)_glia
1,2,1641.292848,324.423216,268.446736,751.179312,108.664231,0.431885,5530.136976,0.404789,0.056573,...,2595.651825,True,0.632736,0.632736,0.000386,0.001950,30.576,36.504,0.837607,2018-0153_OS_Pie8_72F_mid-peri_Z-stack_(1)_glia
2,4,963.656928,213.086016,202.444633,432.694080,93.148005,0.492463,3241.019808,0.460116,0.065336,...,2450.736653,True,0.413712,0.413712,0.000429,0.001942,17.940,36.036,0.497835,2018-0153_OS_Pie8_72F_mid-peri_Z-stack_(1)_glia
3,9,585.086112,150.347808,161.802271,289.963440,82.596226,0.518506,1671.128784,0.510476,0.072167,...,2772.324640,True,0.486720,0.486720,0.000832,0.003237,11.076,35.568,0.311404,2018-0153_OS_Pie8_72F_mid-peri_Z-stack_(1)_glia
4,10,1385.667504,267.209280,238.828746,573.112800,96.675192,0.466242,4880.949840,0.404789,0.058869,...,2381.554418,True,0.705744,0.632736,0.000509,0.002368,28.548,29.796,0.958115,2018-0153_OS_Pie8_72F_mid-peri_Z-stack_(1)_glia
5,11,1005.952896,196.196832,141.523059,377.767728,102.998966,0.519358,2553.235776,0.727789,0.123097,...,2208.240759,True,0.219024,0.219024,0.000218,0.001116,10.608,45.552,0.232877,2018-0153_OS_Pie8_72F_mid-peri_Z-stack_(1)_glia
6,12,1209.693888,260.881920,184.262630,707.958576,117.246133,0.368499,4724.055648,0.636299,0.096556,...,2293.355416,True,0.121680,0.121680,0.000101,0.000466,30.420,42.276,0.719557,2018-0153_OS_Pie8_72F_mid-peri_Z-stack_(1)_glia
7,13,1641.438864,288.454608,259.659895,765.707904,111.910776,0.376716,7211.657232,0.430990,0.053762,...,1983.079660,True,0.413712,0.413712,0.000252,0.001434,30.888,39.780,0.776471,2018-0153_OS_Pie8_72F_mid-peri_Z-stack_(1)_glia
8,21,973.269648,213.572736,165.801099,384.119424,84.282771,0.556006,2679.880320,0.508337,0.097629,...,2671.321956,True,2.336256,1.533168,0.002400,0.007179,17.940,31.512,0.569307,2018-0153_OS_Pie8_72F_mid-peri_Z-stack_(1)_glia
9,22,1356.683328,240.050304,179.975877,469.344096,92.260156,0.511459,4494.469824,0.512625,0.093129,...,2811.688527,True,16.670160,8.030880,0.012287,0.033455,28.236,28.860,0.978378,2018-0153_OS_Pie8_72F_mid-peri_Z-stack_(1)_glia


In [35]:

viewer = napari.Viewer()
viewer.add_image(ch0)
viewer.add_image(result_image)


<Image layer 'result_image' at 0x7fa3b1902890>

In [ ]:


def list_operations(search_term = None):
    ops = cle.operations(search_term)
    for name in ops:
        op = ops[name]
        func = cle.operation(name)

        if hasattr(func, 'fullargspec'):
            print(name + "(" + str(func.fullargspec.args).replace('[','').replace(']','').replace('\'','') + ")")
        else:
            print(name)

list_operations()

